In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math, random
# %matplotlib inline

import torch
from torch import nn
import torch.nn.functional as F  # 避免relu和sigmoid的初始化，可以直接调用

from importlib import reload 
from qflib import basic
reload(basic)

global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

df=pd.DataFrame()  # 初始化空值
# pd.options.display.float_format = '{:.7g}'.format 
vTest = 1000

import torch.backends.cudnn as cudnn
cudnn.benchmark = True  
cudnn.deterministic = True

xl_01dgree = 0.001745 # 0.1度 
xl_02dgree = 0.003490 # 0.2度 
xl_03dgree = 0.005230 # 0.3度
xl_dgree = xl_03dgree

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print( device )
# if torch.cuda.is_available():
#     torch.cuda.caching_allocator_delete
#     torch.cuda.empty_cache()  # 释放显存
#     print('Memory Allocated', torch.cuda.memory_allocated() )

cuda:0


In [64]:
# 读取数据 
sql="SELECT * FROM ds_qf60"
# sql="SELECT * FROM ds_qf60_2"
df = pd.read_sql_query(sql, conn, index_col=None)
del df['index']

In [65]:
""" 确定训练的目标 """
df['class0'] = df['test00']
# df['class0'] = df['test5_8']
# df['class0'] = df['test10_10']
# df['class0'] = df['test20_20']
# df['class0'] = df['test20_30']
# df.info()
len(df)

101148

In [66]:
df.class0.unique(), df.class0.value_counts(), df.class0.value_counts()/len(df)

(array([1., 0.]),
 1.0    51474
 0.0    49674
 Name: class0, dtype: int64,
 1.0    0.508898
 0.0    0.491102
 Name: class0, dtype: float64)

In [67]:
# basic.angle2degree(1)

In [68]:
# 字段优化处理
df['jx_dg_120'] = df['jx_xl_120'].apply(np.arctan).apply(np.degrees) * 100
df['jx_dg_60'] = df['jx_xl_60'].apply(np.arctan).apply(np.degrees)  * 100
df['jx_dg_20'] = df['jx_xl_20'].apply(np.arctan).apply(np.degrees)  * 100


In [69]:
df['pct_fl_3_10']  = (df['lj_fl_3_10'] - 1) * 3/2  * 100
df['pct_fl_5_20']  = (df['lj_fl_5_20'] - 1) *20/15 * 100
df['pct_fl_20_60'] = (df['lj_fl_20_60'] -1) *60/40 * 100
# df['zs_20'] = df['jx_zs_20'] > 0, 
df['zs_20'] = (df['jx_zs_20'] > 0).astype(int)

In [70]:
df[ ['jx_dg_60', 'jx_xl_60', 'pct_fl_5_20','lj_fl_5_20'] ].head(10)

,jx_dg_60,jx_xl_60,pct_fl_5_20,lj_fl_5_20
0,-15.009970,-0.002620,58.877333,1.44158
1,-14.688314,-0.002564,57.585333,1.43189
2,-14.228117,-0.002483,37.545333,1.28159
3,6.763592,0.001180,76.026667,1.57020
4,-12.851937,-0.002243,23.410667,1.17558
5,0.102731,0.000018,47.942667,1.35957
6,2.157392,0.000377,38.078667,1.28559
7,-1.541577,-0.000269,16.490667,1.12368
8,-1.698121,-0.000296,18.222667,1.13667
9,-0.363775,-0.000063,21.672000,1.16254


In [71]:
#  'diff', 'dea',, 
# X_columns0 = ['pct_change', 
#              'jx_days_ud60', 
#              'jx_xl_120','jx_xl_60', 'jx_xl_20', 
#              'jx_zs_20', 
#              'lj_fl_5_20','lj_fl_20_60']
X_columns = ['jx_dg_120','jx_dg_60', 'jx_dg_20', 
             'pct_change', 
             'pct_fl_3_10','pct_fl_5_20','pct_fl_20_60']

In [72]:
df[ X_columns ]

,jx_dg_120,jx_dg_60,jx_dg_20,pct_change,pct_fl_3_10,pct_fl_5_20,pct_fl_20_60
0,8.163211,-15.009970,3.387074,-1.1538,47.08950,58.877333,-20.23005
1,7.001770,-14.688314,9.715350,0.5414,69.82050,57.585333,-21.35475
2,6.349688,-14.228117,4.073804,1.8913,66.55650,37.545333,-24.00660
3,-10.723981,6.763592,0.599789,4.3909,52.57350,76.026667,3.87000
4,-1.070829,-12.851937,0.507476,0.1154,35.79900,23.410667,25.59600
...,...,...,...,...,...,...,...
101143,-12.370827,-5.602185,6.317831,2.0846,26.41800,2.558667,-28.27590
101144,0.000000,-14.670610,3.767122,-5.1149,-4.66905,3.510667,-63.25680
101145,0.000000,-13.477604,25.466546,-5.5798,-8.51835,13.884000,-64.49850
101146,0.000000,-9.332099,47.549478,0.2736,-3.76890,0.613333,-66.76395


In [73]:
X_data = df[X_columns]  
# X = torch.from_numpy(X_data.values).type(torch.float32)
Y_data = df.class0.values.reshape(-1,1)  # 转换成 pd type array
# Y = torch.from_numpy(Y_data).type(torch.float32)
X_data.shape, X_data.head(3)
# , Y.shape

((101148, 7),
    jx_dg_120   jx_dg_60  jx_dg_20  pct_change  pct_fl_3_10  pct_fl_5_20  \
 0   8.163211 -15.009970  3.387074     -1.1538      47.0895    58.877333   
 1   7.001770 -14.688314  9.715350      0.5414      69.8205    57.585333   
 2   6.349688 -14.228117  4.073804      1.8913      66.5565    37.545333   
 
    pct_fl_20_60  
 0     -20.23005  
 1     -21.35475  
 2     -24.00660  )

In [74]:
unique, counts = np.unique(Y_data, return_counts=True)
unique, counts


(array([0., 1.]), array([49674, 51474], dtype=int64))

In [75]:
import torch.nn.parameter as param

class Model3(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(7, 32)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(32, 16)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(16, 1)   # 接着，再定义64个后续中间层
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.sigmoid( self.liner_3(x) )
        return x

# 更加高级语言法，等价于Model
# class Model_high(nn.Module):
#     def __init__(self):
#         super().__init__()      # 继承父类中所有的属性
#         self.liner_1 = nn.Linear(29, 64)   # X.size(1)= 20； 定义64个后续中间层
#         self.liner_2 = nn.Linear(64, 32)   # 接着，再定义64个后续中间层
#         self.liner_4 = nn.Linear(32, 1)   # 接着，再定义64个后续中间层
#         self.relu = nn.ReLU()
#         self.sigmoid = nn.Sigmoid()
#     def forward(self, input):
#         x = self.Liner_1(input)
#         x = self.relu(x)
#         x = self.Liner_2(x)
#         x = self.relu(x)
#         x = self.Liner_3(x)
#         x = self.sigmoid(x)
#         return x

In [97]:
# Ver 4.2 - sklearn mode

lr = 0.001
# lr = 0.00001
def get_model():
    model = Model3()
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    return model, opt

model, optim = get_model()
model = model
# model, optim 

loss_fn = nn.BCELoss()
# loss_fn = loss_fn.to(device)
batch = 2
# batch = 4
# batch = 32
# batch = 1024
epochs = 20
no_of_batches = len(X_data)//batch
from torch.utils.data import TensorDataset, DataLoader

def accuracy(y_pred, y_true):
    y_pred = (y_pred > 0.5).type(torch.int32)
    acc = (y_pred == y_true).float().mean()
    return acc

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X_data, Y_data)

unique, counts = np.unique(Y_data, return_counts=True)
print( 'Y_data: ', dict(zip(unique, counts)) )

train_x = torch.from_numpy(train_x.values).type(torch.float32)
train_y = torch.from_numpy(train_y).type(torch.float32)
train_ds = TensorDataset(train_x, train_y)
train_dl = DataLoader(train_ds, batch_size=batch, shuffle=True)  # 需要shuffle，乱序执行
# unique, counts = np.unique(train_y, return_counts=True)
# print( 'train_y: ', dict(zip(unique, counts)) )

test_x = torch.from_numpy(test_x.values).type(torch.float32)
test_y = torch.from_numpy(test_y).type(torch.float32)
test_ds = TensorDataset(test_x, test_y)
test_dl = DataLoader(test_ds, batch_size=batch, shuffle=False)  # 不需要shuffle，减少运算量

train_x.shape, test_x.shape, train_y.shape, test_y.shape

Y_data:  {0.0: 49674, 1.0: 51474}


(torch.Size([75861, 7]),
 torch.Size([25287, 7]),
 torch.Size([75861, 1]),
 torch.Size([25287, 1]))

In [77]:
unique, counts = np.unique(Y_data, return_counts=True)
unique, counts

(array([0., 1.]), array([49674, 51474], dtype=int64))

计算正确率
- Sigmoid()是0和1之间的值， 大于0.5为1，否则是0
- out1 = (y-pred >0.5).type(torch.int32)  : 转换成0或1
- (out1 == labels).float().mean()

In [98]:
%%time
epochs = 20
for epoch in range( epochs ):
    for x,y in train_dl:
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
    with torch.no_grad():
        # print('epoch: ', epoch) 
        epoch_accuracy = accuracy(model(train_x), train_y)
        epoch_loss = loss_fn(model(train_x), train_y).data.item()
        epoch_test_accuracy = accuracy(model(test_x), test_y)
        epoch_test_loss = loss_fn(model(test_x), test_y).data.item()
        print('epoch: ', 
            epoch, ' train_loss: ', round(epoch_loss,4), ' Acc：', round(epoch_accuracy.item(),4),
            epoch, ' test_loss: ', round(epoch_test_loss,4), ' Acc：', round(epoch_test_accuracy.item(),4) )

epoch:  0  train_loss:  0.6899  Acc： 0.5272 0  test_loss:  0.6903  Acc： 0.5257
epoch:  1  train_loss:  0.6896  Acc： 0.5295 1  test_loss:  0.6903  Acc： 0.5254
epoch:  2  train_loss:  0.689  Acc： 0.5321 2  test_loss:  0.6895  Acc： 0.5294
epoch:  3  train_loss:  0.6884  Acc： 0.5421 3  test_loss:  0.6891  Acc： 0.5382
epoch:  4  train_loss:  0.688  Acc： 0.5359 4  test_loss:  0.6889  Acc： 0.5315
epoch:  5  train_loss:  0.6872  Acc： 0.5411 5  test_loss:  0.6887  Acc： 0.5366
epoch:  6  train_loss:  0.6886  Acc： 0.5476 6  test_loss:  0.6904  Acc： 0.5389
epoch:  7  train_loss:  0.6871  Acc： 0.5436 7  test_loss:  0.6879  Acc： 0.5393
epoch:  8  train_loss:  0.6875  Acc： 0.5411 8  test_loss:  0.6889  Acc： 0.5355
epoch:  9  train_loss:  0.6862  Acc： 0.5467 9  test_loss:  0.688  Acc： 0.5436
epoch:  10  train_loss:  0.6868  Acc： 0.5469 10  test_loss:  0.6885  Acc： 0.5444
epoch:  11  train_loss:  0.6872  Acc： 0.546 11  test_loss:  0.6897  Acc： 0.5399
epoch:  12  train_loss:  0.6856  Acc： 0.5504 12  tes

In [79]:
accuracy(model(test_x), test_y)

tensor(0.5378)

In [80]:
# df.head(1)

In [81]:

df_mn = df[ df['class0']==1.0]
# df_mn = df[ df['class0'] !=1 ]
# df_mn = df_now
# df_mn = df

X_data2 = df_mn[X_columns]  
Y_data2 = df_mn.class0.values.reshape(-1,1)  # 转换成 pd type array
# print( X_data2.shape, Y_data2.shape )

unique, counts = np.unique(Y_data2, return_counts=True)
print( 'Y_data2: ', dict(zip(unique, counts)) )

train_x2, test_x2, train_y2, test_y2 = train_test_split(X_data2, Y_data2, shuffle=True)
# train_x = torch.from_numpy(train_x.values).type(torch.float32).to(device)
# train_y = torch.from_numpy(train_y).type(torch.float32).to(device)
# train_ds = TensorDataset(train_x, train_y)
# train_dl = DataLoader(train_ds, batch_size=batch, shuffle=True)  # 需要shuffle，乱序执行

unique, counts = np.unique(test_y2, return_counts=True)
print( 'test_y2: ', dict(zip(unique, counts)) )

test_X2 = torch.from_numpy(test_x2.values).type(torch.float32)
test_Y2 = torch.from_numpy(test_y2).type(torch.float32)
# unique, counts = np.unique(test_Y2, return_counts=True)
# print( 'test_Y2: ', dict(zip(unique, counts)) )
# test_ds = TensorDataset(test_x, test_y)
# test_dl = DataLoader(test_ds, batch_size=batch, shuffle=False)  # 不需要shuffle，减少运算量
# print( test_X2.shape, test_Y2.shape )
accuracy(model(test_X2), test_Y2)

Y_data2:  {1.0: 51474}
test_y2:  {1.0: 12869}


tensor(0.5657)

In [82]:
# unique, counts = np.unique(Y_data2, return_counts=True)
# print('Y_data: ' dict(zip(unique, counts)) )

In [83]:
# test_Y2
# newY2 = model(test_X2)
# # (newY2 >0.5).type(torch.int32).mean()
# y_pred = (newY2 > 0.5).type(torch.int32)
# acc = (y_pred == 0).float().mean()
# acc

In [84]:
""" 模型保存 """
# torch.save( model, '../model/test5-8.pth')
# path = '../model/test5-8'
# torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optim.state_dict(),
#            }, path)

' 模型保存 '

In [85]:
# list1 = []
# for i in range(0,X_app.size(0)):
#     list1.append( random.randint(1,1) )
# len(list1)
# Y_app = torch.tensor( list1 )
# Y_app.to(device)
# Y_app.reshape(-1,1).size(0) 

In [86]:
""" 手工取数据: 测试(1) """
# column_str = 'ts_code, trade_date, pct_change, diff, dea, bar, jx_days_ud60, \
# jx_xl_250, jx_xl_120, jx_xl_60,\
# jx_xl_20, jx_xl_10, jx_xl_5, \
# jx_zs_5, jx_zs_10, jx_zs_20, jx_zs_60,\
# lj_fl_1_3, lj_fl_3_10, lj_fl_5_20, lj_fl_20_60, \
# sz_max_5, sz_max_10, sz_max_20 '
# # print(column_str)

# sql_new= "SELECT " + column_str +" \
# from day_zhibiao \
# where trade_date>='20230601' and sz_max_10>10 and \
# jx_days_ud60 > 20 and jx_days_ud60 < 60 and \
# jx_xl_60>-"+str(xl_dgree)+" and  \
# jx_xl_20>0 "
# print(sql_new)
# df_new = pd.read_sql_query(sql_new, conn)
# print( df_new.shape )
# # print( df_new[ df_new['sz_max_5']>8 ].head(3) )
# print( df_new[ df_new['sz_max_10']>10 ][['ts_code','trade_date','sz_max_10','sz_max_20']].head(3) )

# X_columns = ['pct_change', 
#              'diff', 'dea', 'bar', 
#              'jx_days_ud60', 
#              'jx_xl_250', 'jx_xl_120','jx_xl_60', 
#              'jx_xl_20', 'jx_xl_10', 'jx_xl_5', 
#              'jx_zs_5', 'jx_zs_10','jx_zs_20', 'jx_zs_60', 
#              'lj_fl_1_3', 'lj_fl_3_10', 'lj_fl_5_20','lj_fl_20_60']
# df_app = df_new[X_columns]
# X_app = torch.from_numpy(df_app.values).type(torch.float32).to(device)

# print( df_app.head(3) )
# print('X_app: ', X_app.shape, X_app.size )

' 手工取数据: 测试(1) '

In [87]:
# list1 = []
# for i in range(0,X_app.size(0)):
#     list1.append( 1.0 )
# Y_app = torch.tensor( list1).type(torch.float32).reshape(-1,1).to(device)
# Y_app.shape


In [88]:
# Y_app = torch.tensor( list1).type(torch.float32)
# X_app.shape, Y_app.shape


In [89]:
# print( '准确率：', accuracy(model(X_app), Y_app) )

In [90]:
# test_y
# Y_app

In [91]:
# X_app
# Y_app.to(device)
# accuracy(model(X_app), Y_app)
# accuracy(model(test_x), test_y)

In [92]:
# Y_app2

In [93]:
# y_pred2 = (Y_app2 > 0.5).type(torch.int32)
# y_pred2

In [94]:
# model = TheModelClass(*args, **kwargs)
# optimizer = TheOptimizerClass(*args, **kwargs)

# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']

数据准确性验证
过拟合/欠拟合
- 过拟合：对于已知数据过度拟合，对未知数据预测性差
- 欠拟合：对于已知数据拟合不够，对未知数据预测性差
  
tensor合并
- 

In [95]:
# train_x.shape, train_y.shape
# train_cat = torch.cat([train_x, train_y],dim=1)   # 沿着Dimension 1进行合并
# train_cat.shape
# train_all.numpy()
# np.savetxt('../dataset/test20.csv', train_cat.numpy(),fmt='%.8f',delimiter=',') # 直接覆盖
# train_np = np.loadtxt('../dataset/test20.csv', dtype = np.float32, delimiter=',')

In [96]:
# Ver 4.1 - random_split Mode， charlie
# from torch.utils.data import random_split    
# def accuracy(y_pred, y_true):
#     y_pred = (y_pred > 0.5).type(torch.int32)
#     acc = (y_pred == y_true).float().mean()
#     return acc

# HR_dataset = TensorDataset(X, Y)
# test_ds, test_ds = random_split(HR_dataset, [0.8, 0.2]) 
# HR_dataloader = DataLoader(test_ds, batch_size=batch, shuffle=True)  # shuffle乱序取数
# epochMode = 41
# if epochMode == 41:  
#     for epoch in range( epochs ):
#         # for x,y in HR_dataloader:
#         for x,y in HR_dataloader:
#             y_pred = model(x)
#             loss = loss_fn(y_pred, y)
#             optim.zero_grad()
#             loss.backward()
#             optim.step()
#         with torch.no_grad():
#             print('epoch: ', epoch, ' loss: ', loss_fn(model(X), Y).data.item() )

Claude: 根据提供的notebook,Claude2对代码的建议和注释如下:
1. 数据处理部分可以添加更多注释,说明读取的数据字段含义,以及对数据的检查和处理思路。
2. 模型定义部分可以添加注释,说明输入和输出维度的设定依据,网络层数和神经元数量的选择依据等。
3. 训练评估部分的注释可以更详细,说明采用的数据分割方式,训练过程中的超参数,以及评估指标的计算方法。
4. 模型训练结果可以进行更深入的分析,检查是否存在过拟合或欠拟合,并结合业务目标提出后续优化思路。 
5. 可以添加模型效果可视化的代码,如Loss曲线,预测结果示例等,更直观地呈现模型效果。
6. 添加模型保存和加载的代码,便于训练后的模型重用。
7. 可以尝试不同模型结构或训练策略的比较实验。
8. 可以基于代码封装出函数或类,提高代码的复用性和模块化程度。
9. 添加类型注释,可以提高代码可读性和健壮性。
总体而言,该notebook内容完整,但仍有进一步提升注释、可视化、模块化、模型分析等方面的优化空间。